In [5]:
!pip install ortools

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 4.5 MB/s eta 0:00:00a 0:00:01
Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3


In [14]:
from ortools.linear_solver import pywraplp
import pandas as pd


In [7]:
tasks = pd.read_csv('../data/tasks.csv')
crew = pd.read_csv('../data/crew.csv')

# Convert availability to boolean
crew['Availability'] = crew['Availability'].map({'True': True, 'False': False})

In [15]:
solver = pywraplp.Solver.CreateSolver('SCIP')

# Maximum tasks per crew member
MAX_TASKS_PER_CREW = 3

In [16]:
assignments = {}
for _, task in tasks.iterrows():
    for _, crew_member in crew.iterrows():
        assignments[(task['Task_ID'], crew_member['Crew_ID'])] = solver.BoolVar(
            f"assign_t{task['Task_ID']}_c{crew_member['Crew_ID']}")

In [17]:
# Objective: Minimize total assignments while balancing workload
objective = solver.Objective()
for (task_id, crew_id), var in assignments.items():
    # Get crew availability
    crew_available = crew[crew['Crew_ID'] == crew_id]['Availability'].iloc[0]
    # Penalize assigning tasks to unavailable crew
    coeff = 1000 if not crew_available else 1
    objective.SetCoefficient(var, coeff)
objective.SetMinimization()

In [18]:
# Constraint 1: Each task must be assigned to exactly one crew member
for _, task in tasks.iterrows():
    solver.Add(
        sum(assignments[(task['Task_ID'], crew_member['Crew_ID'])]
            for _, crew_member in crew.iterrows()) == 1)

# Constraint 2: Limit maximum tasks per crew member
for _, crew_member in crew.iterrows():
    solver.Add(
        sum(assignments[(task['Task_ID'], crew_member['Crew_ID'])]
            for _, task in tasks.iterrows()) <= MAX_TASKS_PER_CREW)

# Constraint 3: Don't assign tasks to unavailable crew
for _, crew_member in crew.iterrows():
    if not crew_member['Availability']:
        for _, task in tasks.iterrows():
            solver.Add(assignments[(task['Task_ID'], crew_member['Crew_ID'])] == 0)

In [19]:
status = solver.Solve()

In [20]:
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    results = []
    for (task_id, crew_id), var in assignments.items():
        if var.solution_value() > 0:
            results.append({"Task ID": task_id, "Crew ID": crew_id})
    results_df = pd.DataFrame(results)
    results_df.to_csv('../data/Optimal_Schedule.csv', index=False)
else:
    print("No optimal solution found.")

Optimal solution found!
